In [44]:
# import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt

from data_structures import Signal, Portfolio

### Understanding Price Movements

In [ ]:
df = pd.read_excel(r'C:\Users\MarcoHui\Desktop\SimulatorData 50ATS OLD VERSION 35.xlsx', sheet_name='prices', skiprows=1, usecols="A,E,K:CRO")

# reduce signals to improve plot readability
df = df.head(50)

In [ ]:
value_vars = df.columns[3:]
temp = pd.melt(df, id_vars='SourceDateNam', value_vars=value_vars, var_name='Days After Signal Date', value_name='Price')

fig, ax = plt.subplots(figsize=(80, 10))

plot = sns.lineplot(
    data=temp,
    x='Days After Signal Date',
    y='Price',
    hue='SourceDateNam',
    ax=ax
)

### Testing Portfolio Simulator Mechanisms

In [43]:
# set up
temp = df[46:49]
my_portfolio = Portfolio(5)

# buy signals
my_portfolio.buyFromDf(temp, 1)

# check that they have been bought
print('\nAFTER FIRST BUY')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))


# update price and value
my_portfolio.updateMonthlyPrice()
my_portfolio.updateMonthlyVal()

# check that price and values have been updated
print('\nAFTER ONE PRICES AND VALUES UPDATE')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# update price and value until expiry
for i in range(157):
    my_portfolio.updateMonthlyPrice()
    my_portfolio.updateMonthlyVal()

# check that price and values have been updated
print('\nAFTER MANY PRICES AND VALUES UPDATES')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# dump expired
my_portfolio.dumpExpired()

# print sell list
print('\nWHAT WE DUMPED')
print(my_portfolio.solddf)

# check that expired signals have been sold at last available price
print('\nAFTER EXPIRED DUMPED')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# buy more signals
temp = df[49:50]
my_portfolio.buyFromDf(temp, 4)

# check that cash is gone and oldest stock has been replaced
print('\nAFTER ANOTHER BUY')
my_portfolio.display()
print('cash: ', str(my_portfolio.cash))

# print sell list
print('\nWHAT WE SOLD TO FUND NEW SIGNAL')
print(my_portfolio.solddf)


AFTER FIRST BUY
jna34139817The    pricelast: na    priceprev/now/next: 33.71921 33.83364 33.66199    growth: 0    val: 1
zach72139818thi    pricelast: 10.45    priceprev/now/next: 5.9 5.98 5.65    growth: 0    val: 1
jna34139819GOO    pricelast: na    priceprev/now/next: 8.3515 8.05025 8.12975    growth: 0    val: 1
cash:  2

AFTER ONE PRICES AND VALUES UPDATE
jna34139817The    pricelast: na    priceprev/now/next: 33.83364 33.66199 33.29962    growth: -0.005073353029706518    val: 0.9949266469702935
zach72139818thi    pricelast: 10.45    priceprev/now/next: 5.98 5.65 8.34    growth: -0.05518394648829432    val: 0.9448160535117057
jna34139819GOO    pricelast: na    priceprev/now/next: 8.05025 8.12975 7.87675    growth: 0.009875469705909688    val: 1.0098754697059096
cash:  2

AFTER MANY PRICES AND VALUES UPDATES
jna34139817The    pricelast: na    priceprev/now/next: 39.92712 39.85084 39.85084    growth: -0.0019104808961929666    val: 1.1778466638528993
zach72139818thi    pricelast: 10.